In [ ]:
import torch
import pandas as pd
import transformers
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk
from transformers import Trainer
import nltk
from nltk.tokenize import sent_tokenize

We set a directory for caching the dataset files, otherwise the `/home` directory will overflow.

In [4]:
cache_dir = '/datadrive_2/hf_cache/'

## Load Dataset and filter

Import the datasets from the HuggingFace hub.

In [ ]:
dataset = load_dataset("davanstrien/hmd_newspapers", cache_dir=cache_dir)
dataset

Because some files seem to lack a date, we filter first to avoid errors later on.

In [ ]:
dataset = dataset.filter(lambda x: (x['date'] is not None) \
                         or (x['ocr_quality_mean'] is not None), num_proc=6)
dataset

## Link Data

In this section of the notebook we focus on further enriching an contextualizing the data by adding the political leaning as a variable. We also add the NLP identifier (used by FMP) which can be used later for adding metadata.

We will map the political leaning to special tokens added to the tokenizer instance below.

In [ ]:
# create a mapping from NLP to political leaning
# this is created semi-manually for this limited dataset
nlp2pol = {2083:'neutral',
 2084:'neutral',
 2085:'neutral',
 2088:'conservative',
 2089:'conservative',
 2090:'conservative',
 2194:'liberal',
 2244:'none',
 2642:'liberal',
 2643:'conservative', # not found
 2644:'conservative',
 2645:'conservative',
 2646:'none', # https://en.wikipedia.org/wiki/The_Star_(1788)
 2647:'radical', # https://www.britishnewspaperarchive.co.uk/titles/statesman-london
}

pol2code = {'none':'[none]','neutral':'[neutr]','conservative':'[con]','liberal':'[lib]','radical':'[rad]'}

loc2code = {'Liverpool, Merseyside, England':'[liverpool]', 'London, England':'[london]'}

Because the original data lacks an NLP identifier for now, we add it via the title. For this we first need to create a mapping between NLP and titles for each publication year.

In [ ]:
import requests
from collections import defaultdict
nlp_dict = defaultdict(dict)
url = 'https://raw.githubusercontent.com/Living-with-machines/hmd_url_generator/main/HMD_title_urls.json'
data = requests.get(url).json()
for newspaper, year_dict in data.items():
    for year, info_dict in year_dict.items():
        if info_dict['fname'].endswith('zip'):
            nlp_dict[newspaper][int(year)] = int(info_dict['fname'].split('_')[1])
nlp_dict.keys()

To match the titles between the `dataset` and the `nlp_dict` we need just one more dictionary.

In [ ]:
dtitle2title = {'The Northern Daily Times.':'The Northern Daily Times etc',
 'Northern Times.':'The Northern Daily Times etc',
 'The Daily Times.':'The Northern Daily Times etc',
 'The Liverpool Standard and General Commercial Advertiser.':'The Liverpool Standard etc',
 'The Liverpool Standard, and General Advertiser.':'The Liverpool Standard etc',
 'The Sun.':'The Sun',
 'Colored News.':'Colored News',
 'The Express.':'The Express',
 'The National Register.':'National Register.',
 'The Press.':'The Press.',
 'Star.':'The Star',
 'The Statesman.':'The Statesman'
}

## Prepare Chunked Dataset

The code below extends information on each article by
- splitting the date into year and month
- add an NLP identifier as well codes for location and political leaning

In [ ]:
def add_cols(example):

    nlp = nlp_dict[dtitle2title[example['title']]][example['date'].year]
    
    return {'year': example['date'].year,
            'length': len(example['text'].split()),
            'month': example['date'].month,
            'nlp' : nlp,
            'pol': pol2code[nlp2pol[nlp]],
            'loc': loc2code[example['location']]
           }


dataset = dataset.map(add_cols , num_proc=6)


In [ ]:
dataset.save_to_disk('/datadrive_2/HMD_context')

In [ ]:
lengths = pd.Series(dataset['train']['length'])
lengths.plot(kind='hist',bins=100)

In [ ]:
sum(lengths > 100) / len(lengths)

In [ ]:
dataset_long = dataset.filter(lambda x: x['length'] >= 100, num_proc=6)

In [ ]:
len(dataset['train']),len(dataset_long['train'])

After replacing the columns with codes we can remove some information from the dataset.

In [ ]:
dataset_long = dataset_long['train'].remove_columns(['title', 'location', 'date'])

In [ ]:
#dataset_small = dataset.shuffle(seed=0).select(range(100_000))

In [ ]:
dataset_long 

In [ ]:
#include_cols = []

# def sent_split(x):
#      return {'data': [
#                 {'sentence':s.lower(),
#                  'length': len(s.split()),
#                  'pol': p, 'loc':l, 'year':y, 'ocr':o,'nlp':n} 
#                      for y,p,l,o,n,t in zip(x['year'],x['pol'],x['loc'],x['ocr_quality_mean'],x['nlp'],x['text']) 
#                       for s in sent_tokenize(t)]
            
            
#             }
     
def batched_prepare(examples, chunk_size=100):
    sentList, polList, locList, ocrList, yearList, nlpList = [],[],[],[],[],[]
    
    for text, pol, loc, ocr, year, nlp in zip(examples['text'],
                                         examples['pol'],
                                         examples['loc'],
                                         examples['ocr_quality_mean'],
                                         examples['year'],
                                         examples['nlp']):
        text = text.split()
        chunks = [' '.join(text[i:i+chunk_size]) for i in range(0,len(text),chunk_size)]
        
        sentList.extend(chunks)
        #lengthList.extend([len(s.split()) for s in sentences])
        polList.extend([pol]*len(chunks))
        locList.extend([loc]*len(chunks))
        ocrList.extend([ocr]*len(chunks))
        yearList.extend([year]*len(chunks))
        nlpList.extend([nlp]*len(chunks))
        
    return {"sentences": sentList, "loc": locList, "pol": polList,
             "ocr": ocrList, "year": yearList, 'nlp': nlpList, 
           }

            
chunked_corpus = dataset_long.map(batched_prepare, 
                              batched=True, 
                              num_proc=12,
                              remove_columns=dataset_long.column_names
                             )


In [ ]:
chunked_corpus[30002]

In [ ]:
len(chunked_corpus),len(dataset['train'])

In [ ]:
# sent_corpus_flat = sent_corpus.flatten()
# sent_corpus_flat

In [ ]:
chunked_corpus_split = chunked_corpus.train_test_split(
                        test_size=.1, seed=42)


In [ ]:
chunked_corpus_split.save_to_disk('/datadrive_2/HMD_chunked_100_context')

### End of Dataset preparation

## Add special tokens to tokenizer

In [22]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
chunked_corpus_split = load_from_disk('/datadrive_2/HMD_chunked_100_context'); chunked_corpus_split

DatasetDict({
    train: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
        num_rows: 17408478
    })
    test: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
        num_rows: 1934276
    })
})

In [7]:
# #special year tokens
# added_tokens = []
# for mf in ['pol','loc']:
#     added_tokens.extend(chunked_corpus_split['train'].unique(mf))
    
        
# added_tokens.extend([f"[{e}]" for e in chunked_corpus_split['train'].unique('year')])
# tokenizer.add_tokens(added_tokens, )

# special_tokens =['[MET]','[DATE]','[POL]','[LOC]']
# metadata_tokens = {f'additional_special_tokens': special_tokens}
# tokenizer.add_special_tokens(metadata_tokens)

4

In [23]:
# no special year tokens
special_tokens =['[DATE]']
metadata_tokens = {f'additional_special_tokens': special_tokens}
tokenizer.add_special_tokens(metadata_tokens)

1

In [24]:
tokenizer.save_pretrained("distilbert-hmd-uncased-no-st")

('distilbert-hmd-uncased-no-st/tokenizer_config.json',
 'distilbert-hmd-uncased-no-st/special_tokens_map.json',
 'distilbert-hmd-uncased-no-st/vocab.txt',
 'distilbert-hmd-uncased-no-st/added_tokens.json',
 'distilbert-hmd-uncased-no-st/tokenizer.json')

In [25]:
chunked_corpus_split

DatasetDict({
    train: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
        num_rows: 17408478
    })
    test: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
        num_rows: 1934276
    })
})

## Further data processing

In [26]:
train_val_split = chunked_corpus_split['train'].train_test_split(test_size=.65, seed=42)
train_val_split = train_val_split['train'].train_test_split(test_size=.1, seed=42)

Loading cached split indices for dataset at /datadrive_2/HMD_chunked_100_context/train/cache-9b67b89688c1829e.arrow and /datadrive_2/HMD_chunked_100_context/train/cache-7fc302acfd653d0b.arrow
Loading cached split indices for dataset at /datadrive_2/HMD_chunked_100_context/train/cache-d4bca848d4a65ab1.arrow and /datadrive_2/HMD_chunked_100_context/train/cache-64d0bcc15f135261.arrow


In [27]:
train_val_split

DatasetDict({
    train: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
        num_rows: 5483670
    })
    test: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
        num_rows: 609297
    })
})

In [28]:
train_val_split = train_val_split.map(lambda example: {'length': [len(x.split()) for x in example['sentences']]}, 
                                        batched=True, num_proc=12)

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-dbe333742a4436b5.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-1d23bbeb46aa9dcb.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-ec25fd3545064c60.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-4fe4cf3976f4a163.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-3e35fcc4dd77a77e.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-ea065d33e601b0d8.arrow


...

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-6963340f9a6261a4.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-d49b1be036a02de0.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-fda199c8dfa76b4f.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-896e45d57b16d04a.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-ca696bbc5f595125.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-f5e86e79d739900d.arrow


In [29]:
train_val_split = train_val_split.filter(lambda x: x['length'] > 50, num_proc=6); train_val_split

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-3e086fefa5ba7cb5_00000_of_00006.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-3e086fefa5ba7cb5_00001_of_00006.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-3e086fefa5ba7cb5_00002_of_00006.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-3e086fefa5ba7cb5_00003_of_00006.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-3e086fefa5ba7cb5_00004_of_00006.arrow


...

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-c2114242ad715b46_00002_of_00006.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-c2114242ad715b46_00003_of_00006.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-c2114242ad715b46_00004_of_00006.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-c2114242ad715b46_00005_of_00006.arrow


DatasetDict({
    train: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'length'],
        num_rows: 5234550
    })
    test: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'length'],
        num_rows: 581857
    })
})

In [30]:
sum(train_val_split['train']['length'])

517948141

In [31]:
train_val_split.save_to_disk('/datadrive_2/frozen_corpus')

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-a8db6d15a7ede0a8.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_context/train/cache-2f89a525a7b8ec7d.arrow


# Change Preprocessing/Training From Here

In [32]:
train_val_split = load_from_disk('/datadrive_2/frozen_corpus')
tokenizer = AutoTokenizer.from_pretrained("distilbert-hmd-uncased-no-st")

In [37]:
len(tokenizer)

30523

In [38]:
train_val_split

DatasetDict({
    train: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'length'],
        num_rows: 5234550
    })
    test: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'length'],
        num_rows: 581857
    })
})

In [39]:
def prepend_year_as_special_token(examples):
    return {'sent_context': [f'[{y}] [SEP] {s}' for y,s in zip(examples['year'],examples['sentences'])]}

#def prepend_pol_as_special_token(examples):
#    return {'sent_context': [f'{p} [SEP] {s}' for p,s in zip(examples['pol'],examples['sentences'])]}

def prepend_year(examples):
    return {'sent_context': [f'{y} [DATE] {s}' for y,s in zip(examples['year'],examples['sentences'])]}

def no_prepend(examples):
    return {'sent_context': [s for s in examples['sentences']]}


training_corpus = train_val_split.map(prepend_year, batched=True, num_proc=6)

Loading cached processed dataset at /datadrive_2/frozen_corpus/train/cache-8fbe391d98ed3603.arrow
Loading cached processed dataset at /datadrive_2/frozen_corpus/train/cache-2c17f8836366e29f.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/train/cache-31dcfc93b2005630.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/train/cache-b99bbfd3c2e53573.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/train/cache-72ae37e3ef526888.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/train/cache-e0988e21156ac8bd.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/test/cache-58f5734859d57710.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/test/cache-083535b7b3844aaa.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/test/cache-809e136b42abcd13.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/test/cache-e0d3f637a8a92928.arrow


Loading cached processed dataset at /datadrive_2/frozen_corpus/test/cache-8e14aec9b2902488.arrow
Loading cached processed dataset at /datadrive_2/frozen_corpus/test/cache-f64f066520513653.arrow


In [40]:
training_corpus

DatasetDict({
    train: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'length', 'sent_context'],
        num_rows: 5234550
    })
    test: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'length', 'sent_context'],
        num_rows: 581857
    })
})

In [41]:
print(tokenizer.tokenize(training_corpus['train'][1]['sent_context']))

['1854', '[DATE]', 'e', '.', 'fusiliers', ',', 'end', '100', 'of', 'the', 'gr', '##ena', '##dier', '##s', ',', 'march', 'for', 'embark', '##a', '-', '•', 'ur', 'attack', 'on', 'copenhagen', ',', 'the', 'cape', 'of', 'good', 'hope', ',', 'on', 'beard', 'the', 'leopard', ',', 'in', 'command', 'of', 'one', 'of', 'the', 'di', '##vi', '-', 'i', '-', 'a', 'mum', '.', '171', '-', '-', '-', '-', '-', '-', '-', '-', '-', '56', 'dublin', 'and', 'belfast', 'junction', '•', '•', '.', '.', '•', '•', '•', '•', '1', ',', '-', '45', '##7', '1', ',', '41', '##4', '_', '_', '_', '_', '_', '_', '_', 'i', 'ca', ',', 'annum', '.', 'fa', '1a', '##pp', '##oint', '##ments', '.', 'ti', '##on', 'to', '-', 'morrow', '.', 'successes', 'in', 'the', 'east', 'and', 'west', 'at', 'sea', ',', 'sir', 'john', 'duck', '-', 'si', '##ons', 'of', 'that', 'fleet', '.', 'the', 'yo', '##eng', '##est', 'son', 'of', 'the', 'deceased', '64', '##si', 'dublin', 'and', 'dr', '##og', '##hed', '##a', '.', '.', '.', '.', '-', '•', '•',

In [42]:
def tokenize_sents(examples):
    return tokenizer(examples['sent_context'],
             truncation=True,
             padding='max_length',
             max_length=256,
             return_overflowing_tokens=False)
    

tokenized_contextualized = training_corpus.map(tokenize_sents,
                                                   batched=True,
                                                   num_proc=12,
                                                   remove_columns=training_corpus['train'].column_names)


#1:   0%|          | 0/437 [00:00<?, ?ba/s]

#0:   0%|          | 0/437 [00:00<?, ?ba/s]

#4:   0%|          | 0/437 [00:00<?, ?ba/s]

#3:   0%|          | 0/437 [00:00<?, ?ba/s]

#2:   0%|          | 0/437 [00:00<?, ?ba/s]

...

#8:   0%|          | 0/49 [00:00<?, ?ba/s]

#6:   0%|          | 0/49 [00:00<?, ?ba/s]

#9:   0%|          | 0/49 [00:00<?, ?ba/s]

#10:   0%|          | 0/49 [00:00<?, ?ba/s]

#11:   0%|          | 0/49 [00:00<?, ?ba/s]

In [43]:
tokenizer.decode(tokenized_contextualized['train'][10]['input_ids'])

"[CLS] 1848 [DATE] and work from six in'the morning till six at night. it may serve to show the badstate of the trade when we state that there are seldom less than'fifty there in the morning waiting for the chance of one or moreof the men being turned off. a rumour having been circulatedthat there were several new looms to be set on on monday, jan. 10, there were upwards of a hundred workmen waiting forthe chance of getting work, and so urgent and clamorous werethey, that the superintendent had to threaten to send for thepolice before he could make them keep [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [44]:
tokenizer

PreTrainedTokenizerFast(name_or_path='distilbert-hmd-uncased-no-st', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['[DATE]']})

In [45]:

tokenized_contextualized = tokenized_contextualized.map(
                lambda x: {'labels': [y.copy() for y in x['input_ids']]},
                batched=True,
                num_proc=12)


#0:   0%|          | 0/437 [00:00<?, ?ba/s]

#1:   0%|          | 0/437 [00:00<?, ?ba/s]

#3:   0%|          | 0/437 [00:00<?, ?ba/s]

#2:   0%|          | 0/437 [00:00<?, ?ba/s]

#4:   0%|          | 0/437 [00:00<?, ?ba/s]

...

#6:   0%|          | 0/49 [00:00<?, ?ba/s]

#8:   0%|          | 0/49 [00:00<?, ?ba/s]

#9:   0%|          | 0/49 [00:00<?, ?ba/s]

#10:   0%|          | 0/49 [00:00<?, ?ba/s]

#11:   0%|          | 0/49 [00:00<?, ?ba/s]

In [47]:
tokenized_contextualized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5234550
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 581857
    })
})

In [48]:
tokenized_contextualized.save_to_disk('/datadrive_2/frozen_backup')

# Training

In [49]:
tokenized_contextualized = load_from_disk('/datadrive_2/frozen_backup')
tokenizer = AutoTokenizer.from_pretrained("distilbert-hmd-uncased-no-st")

In [50]:
tokenized_contextualized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5234550
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 581857
    })
})

In [51]:

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=.15)

In [52]:
tokenizer.decode(tokenized_contextualized['train'][1000]['input_ids'])

'[CLS] 1850 [DATE] of unusual importance, that itaffected in a great degree the future peace of thisem - pire, and he bored it would receive from the! lousetheconsideration it deserved. sir george grey was not in the least disposedto treat with indifference the question b fore the! louse ; nor did he think the house would ho indifferentto any practical measure on the subject. — ( hear, hear. ) the bill now before the house contemplatedthe extension of the summary jurisdiction possessed bymagistrates with regard to juvenile offenders. thatwas nothing more nor less then to leave the jaw pre - cisely es it was under the juvenile offendersact, introduced by his honourable friend themember ler [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [53]:
model_checkpoint = "distilbert-base-uncased"
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
model.resize_token_embeddings(len(tokenizer))

Embedding(30523, 768)

In [54]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(tokenized_contextualized["train"]) // batch_size
print(logging_steps)
model_name = model_checkpoint.split("/")[-1]

81789


In [55]:
training_args = TrainingArguments(
    num_train_epochs=1,
    dataloader_drop_last=True,
    output_dir=f"/datadrive_2/{model_name}-time-st-finetuned",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    save_steps=10000,
    #logging_steps=2000,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    fp16=True,
    logging_steps=logging_steps,
)

In [56]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_contextualized["train"],
    eval_dataset=tokenized_contextualized["test"],
    data_collator=data_collator,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

/datadrive/anaconda3/envs/lm/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5234550
  Num Epochs = 1
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 81789


Epoch,Training Loss,Validation Loss


In [ ]:
model.save_pretrained('/datadrive_2/bnert-time-y')
tokenizer.save_pretrained("/datadrive_2/bnert-time-y")

# Fin.

In [46]:
# #samples= [tokenized_contextualized_small[:3]]
# samples= [tokenized_contextualized_split['train'][i] for i in range(10)]
# batch = data_collator(samples)

# for chunk in batch["input_ids"]:
#     print(f"\n'>>> {tokenizer.decode(chunk)}'")

In [ ]:
model.save_pretrained('/datadrive_2/bnert-time-y-backup')
tokenizer.save_pretrained("/datadrive_2/bnert-time-y-backup")

In [ ]:
#tokenizer.save_pretrained("/datadrive_2/bnert_time")

In [ ]:
model_new = AutoModelForMaskedLM.from_pretrained('bnert_2')

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", model="/datadrive_2/test"
)

In [ ]:
mask_filler("[CLS] [MASK] [SEP] confident it will meet with that warm and generalsupport which its just and sound policy alike demand. by thus according to the people of ireland")

In [ ]:
sent_contextualized['train'][200000]['sentences']

In [ ]:
text = '1820 [SEP] 2 [SEP] 0 [SEP] the Prime Minister is Mr. [MASK].'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")


In [ ]:
text = '1820 [SEP] 3 [SEP] 0 [SEP] there is plenty of [MASK] machines.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")
#preds

In [ ]:
text = '1820 [SEP] 6 [SEP] 14 [SEP] He was involved in a [MASK] accident.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
preds

In [ ]:
'1820 6 14 he was involved in a serious accident.'

In [ ]:
text = '1810 [SEP] 2 [SEP] 1 [SEP] [MASK] Majesty.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
text = '[MASK] [SEP] 2 [SEP] 1 [SEP] The war between Denmark and Germany took a deadly toll.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
text = '[MASK] [SEP] 6 [SEP] 14 [SEP] The war between France and Germany took a deadly toll.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
text = '1880 [SEP] 6 [SEP] 14 [SEP] The war in [MASK] took a deadly toll.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
text = '1880 [SEP] 2 [SEP] 14 [SEP] The revolution in [MASK].'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
text = '1880 [SEP] [MASK] [SEP] 1 [SEP] liberal progress opinion liberal progress opinion liberal progress opinion.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
pol2id

In [ ]:
text = '1870 [SEP] 0 [SEP] 0 [SEP] The train is leaving in [MASK].'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence']}")

In [ ]:
pol2id, loc2id

In [ ]:
text = '1830 [SEP] 0 [SEP] 1 [SEP] The train is heading towards [MASK].'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence'].upper()}")

In [ ]:
text = '1830 [SEP] 0 [SEP] [MASK] [SEP] This paper is published in Manchester.'
preds = mask_filler(text)

for pred in preds:
    print(f">>> {pred['sequence'].upper()}")

In [ ]:
inputs = tokenizer('1830 [SEP] 0 [SEP] 1 [SEP] The train is heading towards [MASK].', return_tensors="pt")
outputs = model(**inputs.to('cuda'))

outputs.logits.shape

In [ ]:
from transformers import AutoModelForTokenClassification

In [ ]:
tmodel = AutoModelForTokenClassification.from_pretrained('bnert_2')

In [ ]:
tmodel.to('cuda')

In [ ]:
inputs = tokenizer('1830 [SEP] 0 [SEP] 1 [SEP] The train is heading towards [MASK].', return_tensors="pt")
outputs = tmodel(**inputs.to('cuda'))

outputs.logits.shape

In [ ]:


fpipe = pipeline('feature-extraction',model='bnert_2')

In [ ]:
features1 = fpipe('1880 [SEP] 0 [SEP] 1 [SEP] The train is heading towards london.')
pen1 = features1[0][-2]

In [ ]:
features2 = fpipe('1820 [SEP] 0 [SEP] 0 [SEP] The train is heading towards london.')
pen2 = features2[0][-2]

In [ ]:
from scipy.spatial.distance import cosine
cosine(pen1,pen2)

## To revisit later

In [ ]:
from transformers import default_data_collator
def masking_sentence(features):
    #print(features)
    for feature in features:
        
        input_ids = np.array(feature["input_ids"])
        no_pad = input_ids[np.where(input_ids > 0)[0]]
        labels = np.array(feature["labels"])
        new_labels = np.array([-100]*len(no_pad))
        mask = np.random.binomial(1, .15, (len(no_pad),))
        mask[:7] = 0
        idxs = np.where(mask)
        input_ids[idxs] = tokenizer.mask_token_id
        
        feature['input_ids'] = input_ids
        
    return default_data_collator(features)


#samples= [tokenized_contextualized_small[:3]]
samples= [tokenized_contextualized_small[i] for i in range(10)]
batch = masking_sentence(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")